In [1]:
from pathlib import Path
import re

In [78]:
lines = Path('papers.md').read_text(encoding='utf-8-sig').splitlines()
len(lines)

8968

In [79]:
titles = [
    (i, match.group(1))
    for i, line in enumerate(lines)
    if (match := re.fullmatch(r'## (.*)', line))
    and not re.fullmatch(r'[0-9][0-9][0-9][0-9]', match.group(1))
]
titles

[(3458, 'Fine-tuning'),
 (3796, 'Transformers and RNN'),
 (3847, 'OOD'),
 (4049, 'Continual LLM'),
 (4061, 'Other TODO'),
 (4079, 'ASR'),
 (5336, 'CV: localization (detection, segmentation)'),
 (5697, 'CV: few-shot'),
 (5705, 'CV: Image editing'),
 (5737, 'CV: pretraining'),
 (5743, 'SSM, World models with CV, NLP'),
 (5857, 'NLP: interpreting, robustness, uncertainty, shortcuts'),
 (6397, 'Code generation'),
 (6442, 'Fundametals (various)')]

In [81]:
paper_spans: list[tuple[int, int]] = []

def is_note(line: str) -> bool:
    return line.startswith('  ') or line.startswith('    ') or line.startswith('\t')

for i, line in enumerate(lines):
    if re.fullmatch(r'[^ \t].*([0-9][0-9][0-9][0-9].*)\. .*', line):
        j = i + 1
        if len(lines[j].strip()) == 0 and is_note(lines[j + 1]):
            j += 1
        for j in range(j + 1, len(lines)):
            if not is_note(lines[j]) and len(lines[j].strip()) > 0:
                break
        paper_spans.append((i, j))
    elif re.fullmatch(r'@.*', line):
        j = i + 1
        for j in range(i + 1, len(lines)):
            if len(lines[j].strip()) == 0:
                break
            if (
                not lines[j].startswith('\t')
                and not lines[j].strip() == '}'
                and not is_note(lines[j])
            ):
                break
        paper_spans.append((i, j))
    
from itertools import pairwise
for (a1, b1), (a2, b2) in pairwise(paper_spans):
    assert b2 > a2 >= b1 > a1

remaining_lines = lines
for a, b in paper_spans[::-1]:
    remaining_lines = remaining_lines[:a] + remaining_lines[b:]

Path('remaining_lines.txt').write_text('\n'.join(remaining_lines));

In [66]:
re.fullmatch(r'@.*', lines[6795])

<re.Match object; span=(0, 20), match='@article{Qiu2019Nov,'>